# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [0]:
# import libraries
import pickle
import string
import unittest

import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
nltk.download('stopwords')
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.externals import joblib


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
# load data from database
engine = create_engine('sqlite:////content/drive/My Drive/Colab Notebooks/p5/data/DisasterResponse1.db')
df = pd.read_sql_table('messages', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
X = df.message
Y = df.loc[:, 'related':'direct_report']

### 2. Write a tokenization function to process your text data

In [0]:
print(X[0])

Weather update - a cold front from Cuba that could pass over Haiti


In [0]:
def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    clean_tokens = [w for w in clean_tokens if w not in stopwords.words("english")]
    
    return clean_tokens

In [0]:
tokenize(X[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [0]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
pipeline.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [0]:
def perf_report(model, X_test, Y_test):
    '''
    Function to generate classification report on the model
    Input: Model, test set ie X_test & y_test
    Output: Prints the Classification report
    '''
    y_pred = model.predict(X_test)
    for i, col in enumerate(Y_test):
        print(col)
        print(classification_report(Y_test[col], y_pred[:, i]))

In [0]:
perf_report(pipeline, X_test, Y_test) 

related
              precision    recall  f1-score   support

           0       0.67      0.46      0.54      1228
           1       0.85      0.93      0.89      3978

    accuracy                           0.82      5206
   macro avg       0.76      0.69      0.72      5206
weighted avg       0.81      0.82      0.81      5206

request
              precision    recall  f1-score   support

           0       0.89      0.97      0.93      4300
           1       0.76      0.44      0.56       906

    accuracy                           0.88      5206
   macro avg       0.82      0.71      0.74      5206
weighted avg       0.87      0.88      0.86      5206

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5176
           1       0.00      0.00      0.00        30

    accuracy                           0.99      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg       0.99      0.99      0.99      520

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


other_infrastructure
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      4963
           1       0.00      0.00      0.00       243

    accuracy                           0.95      5206
   macro avg       0.48      0.50      0.49      5206
weighted avg       0.91      0.95      0.93      5206

weather_related
              precision    recall  f1-score   support

           0       0.85      0.95      0.90      3712
           1       0.83      0.59      0.69      1494

    accuracy                           0.85      5206
   macro avg       0.84      0.77      0.80      5206
weighted avg       0.85      0.85      0.84      5206

floods
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      4750
           1       0.83      0.23      0.36       456

    accuracy                           0.93      5206
   macro avg       0.88      0.61      0.66      5206
weighted avg       0.92      0

### 6. Improve your model
Use grid search to find better parameters. 

In [0]:
pipeline.get_params()

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        n_estimators='warn',
                                                        n_jobs=Non

In [0]:
parameters =  {'clf__estimator__n_estimators': [50, 100], 
              'clf__estimator__min_samples_split': [2, 4]} 
       

cv = GridSearchCV(pipeline, param_grid=parameters)

In [0]:
cv

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [0]:
cv.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [0]:
cv.best_params_

{'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 100}

In [0]:
perf_report(cv, X_test, Y_test) 

related
              precision    recall  f1-score   support

           0       0.75      0.39      0.52      1228
           1       0.84      0.96      0.89      3978

    accuracy                           0.83      5206
   macro avg       0.80      0.68      0.71      5206
weighted avg       0.82      0.83      0.81      5206

request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      4300
           1       0.83      0.51      0.63       906

    accuracy                           0.90      5206
   macro avg       0.87      0.75      0.79      5206
weighted avg       0.89      0.90      0.89      5206

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5176
           1       0.00      0.00      0.00        30

    accuracy                           0.99      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg       0.99      0.99      0.99      520

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5144
           1       0.00      0.00      0.00        62

    accuracy                           0.99      5206
   macro avg       0.49      0.50      0.50      5206
weighted avg       0.98      0.99      0.98      5206

shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5188
           1       0.00      0.00      0.00        18

    accuracy                           1.00      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg       0.99      1.00      0.99      5206

aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5137
           1       0.00      0.00      0.00        69

    accuracy                           0.99      5206
   macro avg       0.49      0.50      0.50      5206
weighted avg       0.97      0.99      0.98      5206

o

These new parameters resulted in a reduction in performance.

### 8. Try improving your model further. Here are a few ideas:
try other machine learning algorithms
add other features besides the TF-IDF

In [0]:
#Improve  the pipeline
pipeline2 = Pipeline([
    ('best', TruncatedSVD()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [0]:
#Train & predict
pipeline2.fit(X_train, Y_train)
perf_report(pipeline2, X_test, Y_test)

related
              precision    recall  f1-score   support

           0       0.57      0.48      0.52      1563
           1       0.85      0.88      0.86      4991

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.71      0.68      0.69      6554
weighted avg       0.78      0.79      0.78      6554

request
              precision    recall  f1-score   support

           0       0.88      0.98      0.93      5443
           1       0.82      0.36      0.50      1111

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.85      0.67      0.72      6554
weighted avg       0.87      0.88      0.86      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      655

C:\Users\rebecca-barnes\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


other_aid
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5690
           1       0.57      0.04      0.07       864

   micro avg       0.87      0.87      0.87      6554
   macro avg       0.72      0.52      0.50      6554
weighted avg       0.83      0.87      0.82      6554

infrastructure_related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6143
           1       1.00      0.00      0.00       411

   micro avg       0.94      0.94      0.94      6554
   macro avg       0.97      0.50      0.49      6554
weighted avg       0.94      0.94      0.91      6554

transport
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6251
           1       0.40      0.01      0.01       303

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.68      0.50      0.49      6554
weighted avg       0.93      0.

It increase the score and performance with grid search

In [0]:
pipeline2.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text', 'features__word_count', 'features__character_count', 'features__noun_count', 'features__verb_count', 'features__text__analyzer', 'features__text__binary', 'features__text__decode_error', 'features__text__dtype', 'features__text__encoding', 'features__text__input', 'features__text__lowercase', 'features__text__max_df', 'features__text__max_features', 'features__text__min_df', 'features__text__ngram_range', 'features__text__norm', 'features__text__preprocessor', 'features__text__smooth_idf', 'features__text__stop_words', 'features__text__strip_accents', 'features__text__sublinear_tf', 'features__text__token_pattern', 'features__text__tokenizer', 'features__text__use_idf', 'features__text__vocabulary', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_f

In [0]:
#Param tunning 
parameters2 = {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__learning_rate': [1,2] }

In [0]:
cv2 = GridSearchCV(pipeline2, param_grid=parameters2)

In [0]:
cv2.fit(X_train, Y_train)
perf_report(cv2, X_test, Y_test)

This is an improvement over the best model so far. The fact that the best model is associated with the highest available ranges for max depth and number of estimators suggests that the model may be improved by increasing these values.

This appears to be the best model so far, including having predictions for all classes. 

### 9. Export your model as a pickle file

In [0]:
path = '/content/drive/My Drive/Colab Notebooks/p5/data/model1.pkl'
pickle.dump(cv, open(path, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.